In [1]:
library(ggplot2)
library(lubridate)
library(dplyr)
library(jsonlite) 
# Load Data
load_data <- function(file_path) {
  data <- read.csv(file_path, header = TRUE)
  data <- data[-1, ]  # Remove first row if necessary
  data$Extent <- as.numeric(data$Extent)  # Convert Extent to numeric
  data$Date <- as.Date(with(data, paste(Year, Month, Day, sep="-")), format="%Y-%m-%d")
  data$DayOfYear <- yday(data$Date)
  return(data)
}

# Filter Data for Selected Years
filter_data_by_years <- function(data, selected_years) {
  return(data %>% filter(Year %in% selected_years))
}

# Calculate Quantiles for 1981-2010
calculate_quantiles <- function(data) {
  data_median <- data %>% filter(Year >= 1981 & Year <= 2010)
  quantile_data <- data_median %>%
    group_by(DayOfYear) %>%
    summarise(
      MedianExtent = median(Extent, na.rm = TRUE),
      Q1 = quantile(Extent, 0.25, na.rm = TRUE),
      Q3 = quantile(Extent, 0.75, na.rm = TRUE),
      D10 = quantile(Extent, 0.10, na.rm = TRUE),
      D90 = quantile(Extent, 0.90, na.rm = TRUE)
    )
  quantile_data$Year <- "1981-2010"
  return(quantile_data)
}

# Apply Spline Interpolation on Quantile Data
apply_spline_interpolation <- function(quantile_data) {
  spline_quantile <- data.frame(
    DayOfYear = seq(min(quantile_data$DayOfYear), max(quantile_data$DayOfYear), length.out = 100),
    Extent = spline(quantile_data$DayOfYear, quantile_data$MedianExtent, n = 100)$y,
    Q1 = spline(quantile_data$DayOfYear, quantile_data$Q1, n = 100)$y,
    Q3 = spline(quantile_data$DayOfYear, quantile_data$Q3, n = 100)$y,
    D10 = spline(quantile_data$DayOfYear, quantile_data$D10, n = 100)$y,
    D90 = spline(quantile_data$DayOfYear, quantile_data$D90, n = 100)$y
  )
  return(spline_quantile)
}

# Apply Spline Interpolation on Selected Data
apply_spline_on_selected_data <- function(data_filtre) {
  spline_data <- data_filtre %>%
    group_by(Year) %>%
    group_modify(~ {
      spline_interp <- as.data.frame(spline(.x$DayOfYear, .x$Extent, n = 100))
      colnames(spline_interp) <- c("DayOfYear", "Extent")
      spline_interp$Year <- unique(.x$Year)  # Assign the correct Year
      return(spline_interp)
    })
  return(spline_data)
}

# Base Plot Function
create_base_plot <- function(spline_data, day_list, month_list, couleurs_années) {
  return(ggplot(spline_data, aes(x = DayOfYear, y = Extent)) +
           scale_x_continuous(
             expand = c(0, 0),
             breaks = day_list,
             labels = month_list
           ) +
           scale_y_continuous(
             expand = c(0,0),
             limits = c(0,22.5),
             breaks = seq(2.5, 22.5, by = 2.5),
             labels = seq(2.5, 22.5, by = 2.5)
           ) +
           scale_linetype_manual(name = NULL, values = c("Median" = "solid"), guide = guide_legend(order = 1)) +  # Median line first
           scale_fill_manual(name = NULL, values = c("IQR (Q1-Q3)" = "gray25", "D10-D90" = "gray60"), guide = guide_legend(order = 2)) +  # Quantiles second
           scale_color_manual(name = NULL, values = couleurs_années, guide = guide_legend(order = 3)) +  # Years last
           labs(title = "Antarctic Sea Ice Extend",
                subtitle = "(Area of ocean with at least 15% sea ice)",
                x = "Date",
                y = "Extent (Millions of square kilometers)") +
           theme_bw()+
    theme(
      plot.title = element_text(hjust = 0.5, family = "sans", face = "bold", size = 14),
      plot.subtitle = element_text(hjust = 0.5, family = "sans", face = "italic",size = 10),
      axis.title = element_text(family = "sans"),
      axis.text = element_text(family = "sans")
      )
  )
}

# Plot Quantiles on the Base Plot
add_quantiles_to_plot <- function(base_plot, spline_quantile) {
  return(base_plot + 
           geom_line(data = spline_quantile, aes(x = DayOfYear, y = Extent, linetype = "1981-2010 Median"), 
                     color = "black", linewidth = 0.7) +  # Median line
           geom_ribbon(data = spline_quantile, aes(x = DayOfYear, ymin = Q1, ymax = Q3, fill = "Interquartile Range"), 
                       alpha = 0.3) +  # Interquartile range (Q1-Q3)
           geom_ribbon(data = spline_quantile, aes(x = DayOfYear, ymin = D10, ymax = D90, fill = "Interdecile Range"), 
                       alpha = 0.3))  # D10-D90 range
}

# Add Yearly Data Lines to Plot
add_year_lines_to_plot <- function(plot, spline_data) {
  return(plot + geom_line(data = spline_data, aes(color = as.factor(Year)), linewidth = 0.7))  # Lines for selected years
}


#INPUT
json_path <- "galaxy_inputs/galaxy_inputs.json"
json_data <- fromJSON(json_path)

color <- json_data$color
year <- json_data$year
median<-json_data$median

annees_liste <- strsplit(year, ",")[[1]]

if (!is.null(color)) {
  couleurs_liste <- strsplit(color, ",")[[1]]
} else {
  couleurs_liste <- NULL  # Pour éviter les erreurs plus tard
}

do_1981_2010 <- median
# MAIN ----
day_list = c(1,32,61,92,122,153,183,214,245,275,306,336,365)
month_list = c("01_Jan","01_Feb","01_Mar","01_Apr","01_May","01_Jun","01_Jul","01_Aug","01_Sep","01_Oct","01_Nov","01_Dec","31_Dec")

download.file("https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/data/S_seaice_extent_daily_v3.0.csv", "S_seaice_extent_daily_v3.0.csv")
file_path <- "S_seaice_extent_daily_v3.0.csv" #Arg 1 

data <- load_data(file_path)

annees_presentes <- unique(data$Year)



# Si la liste des couleurs est vide, on applique une palette par défaut
if (!exists("couleurs_liste") || is.null(couleurs_liste) || length(couleurs_liste) == 0) {
  if (length(annees_liste) <= 8) {
    # Si la longueur est inférieure ou égale à 8, utiliser la palette 'rainbow'
    couleurs_liste <- rainbow(length(annees_liste))  # Palette rainbow pour max 8 couleurs
  } else {
    # Sinon, utiliser la palette étendue Set1 avec colorRampPalette
    couleurs_liste <- grDevices::colorRampPalette(RColorBrewer::brewer.pal(9, "Set1"))(length(annees_liste))  # Adaptation selon le nombre d'années
  }
}

# Crée un vecteur de couleurs avec les années spécifiées
couleurs_années <- setNames(couleurs_liste, annees_liste)



data_filtre <- filter_data_by_years(data, annees_liste)
quantile_data <- calculate_quantiles(data)
spline_quantile <- apply_spline_interpolation(quantile_data)
spline_data <- apply_spline_on_selected_data(data_filtre)

create_plot <- function(do_1981_2010, spline_data, spline_quantile, day_list, month_list, couleurs_années) {
  base_plot <- create_base_plot(spline_data, day_list, month_list, couleurs_années)
  
  if (do_1981_2010) {
    plot_1981_2010 <- add_quantiles_to_plot(base_plot, spline_quantile)
  } else {
    plot_1981_2010 <- base_plot  # If do_1981_2010 is FALSE, no quantiles are added
  }
  
  final_plot <- add_year_lines_to_plot(plot_1981_2010, spline_data)
  return(final_plot)
}

# Generate and print the plot
plot <- create_plot(do_1981_2010, spline_data, spline_quantile, day_list, month_list, couleurs_années)
png("outputs/fig2.png", width = 20,height = 15,units = "cm",res=1000)
print(plot)
dev.off()



Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




ERROR: Error: lexical error: invalid char in json text.
                                       galaxy_inputs/galaxy_inputs.jso
                     (right here) ------^

